In [116]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.applications import VGG16
from PIL import Image

## Image preprocessing ##

In [3]:
# Load an example image
img_path = '/Users/kamillaraki/Downloads/CROPPED-AMA/1-112-PR-PID-009_2_3.jpg'
img = Image.open(img_path)

# Check the size of the image
print(img.size)


(4855, 4397)


In [4]:
input_shape = (1792, 1792)

In [5]:
input_folder_path = '/Users/kamillaraki/Downloads/CROPPED-AMA'

In [6]:
output_folder_path = '/Users/kamillaraki/Downloads/TEST1'

In [14]:
img_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, rotation_range=0, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, horizontal_flip=True, fill_mode='nearest')


In [16]:
for filename in os.listdir('/Users/kamillaraki/Downloads/CROPPED-AMA'):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Load the image using PIL
        img = Image.open(os.path.join('/Users/kamillaraki/Downloads/CROPPED-AMA', filename))

        # Resize the image to the desired input shape
        img_resized = img.resize(input_shape)

        # Save the resized image to the output folder
        new_filename = filename.split('.')[0] + '_resized.jpg'  # Change the filename as desired
        img_resized.save(os.path.join('/Users/kamillaraki/Downloads/TEST1', new_filename))


In [22]:
input_folder_path = '/Users/kamillaraki/Downloads/CROPPED-UNLAB'

In [23]:
output_folder_path = '/Users/kamillaraki/Downloads/TEST3'

In [24]:
img_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, rotation_range=0, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, horizontal_flip=True, fill_mode='nearest')


In [25]:
for filename in os.listdir('/Users/kamillaraki/Downloads/CROPPED-UNLAB'):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Load the image using PIL
        img = Image.open(os.path.join('/Users/kamillaraki/Downloads/CROPPED-UNLAB', filename))

        # Resize the image to the desired input shape
        img_resized = img.resize(input_shape)

        # Save the resized image to the output folder
        new_filename = filename.split('.')[0] + '_resized.jpg'  # Change the filename as desired
        img_resized.save(os.path.join('/Users/kamillaraki/Downloads/TEST3', new_filename))


## VGG16 ##

In [11]:
# Define the input shape for your images
input_shape = (1792, 1792, 3)

In [12]:
# Define the batch size and number of epochs for training
batch_size = 8
num_epochs = 10

In [13]:
# Define the dataset directory
train_data_dir = '/Users/kamillaraki/Downloads/SAMY'

In [14]:
# Define the path to your labeled dataset for functional P&IDs
train_data_dir_func = '/Users/kamillaraki/Downloads/SAMY/TEST1'

In [15]:
# Define the path to your labeled dataset for dysfunctional P&IDs
train_data_dir_dysfunc = '/Users/kamillaraki/Downloads/SAMY/TEST2'

In [27]:
# Define the path to your unlabeled dataset
test_data_dir = '/Users/kamillaraki/Downloads/SAMY/TESTT3'

In [17]:
# Load the pre-trained VGG16 model without the top layers
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

In [18]:
# Freeze the pre-trained layers to prevent them from being updated during training
for layer in vgg_model.layers:
    layer.trainable = False

In [19]:
# Add new layers to the pre-trained model for your specific problem
model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [20]:
# Compile the model with an appropriate loss function, optimizer, and metrics
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [21]:
# Define the data generators for training, validation, and testing
train_data_gen_func = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_data_gen_dysfunc = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_data_gen = ImageDataGenerator(rescale=1./255)

train_generator = train_data_gen_func.flow_from_directory(train_data_dir, target_size=input_shape[:2], batch_size=batch_size, class_mode='binary', subset='training')
validation_generator = train_data_gen_func.flow_from_directory(train_data_dir, target_size=input_shape[:2], batch_size=batch_size, class_mode='binary', subset='validation')
test_generator = test_data_gen.flow_from_directory(test_data_dir, target_size=input_shape[:2], batch_size=batch_size, class_mode=None, shuffle=False)


Found 29 images belonging to 2 classes.
Found 7 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [22]:
history = model.fit(x=train_generator, steps_per_epoch=train_generator.n//batch_size, epochs=num_epochs, validation_data=validation_generator, validation_steps=validation_generator.n//batch_size)

Epoch 1/10
3/3 [==============================] - 446s 153s/step - loss: 127.0304 - accuracy: 0.4583
Epoch 2/10
3/3 [==============================] - 429s 144s/step - loss: 106.3036 - accuracy: 0.6250
Epoch 3/10
3/3 [==============================] - 374s 111s/step - loss: 76.0322 - accuracy: 0.6667
Epoch 4/10
3/3 [==============================] - 371s 115s/step - loss: 48.8636 - accuracy: 0.8571
Epoch 5/10
3/3 [==============================] - 405s 131s/step - loss: 36.5002 - accuracy: 0.8333
Epoch 6/10
3/3 [==============================] - 408s 133s/step - loss: 6.0778 - accuracy: 0.9583
Epoch 7/10
3/3 [==============================] - 380s 146s/step - loss: 60.7290 - accuracy: 0.8571
Epoch 8/10
3/3 [==============================] - 425s 138s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
3/3 [==============================] - 422s 136s/step - loss: 1.8675e-13 - accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - 349s 109s/step - loss: 53.4590 - accuracy:

In [115]:
model.save('SAMY_VGG16.h5')


## Testing the results on unlabeled dataset ##

In [36]:
test_filenames = os.listdir(test_data_dir)

In [38]:
import pandas as pd
test_df = pd.DataFrame({'TESTT3': test_filenames, 'label': None})


In [40]:
test_data_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_data_gen.flow_from_dataframe(test_df, test_data_dir, x_col='TESTT3', y_col='label', target_size=input_shape[:2], batch_size=batch_size, class_mode=None, shuffle=False)

Found 19 validated image filenames.


In [41]:
preds = model.predict_generator(test_generator, steps=test_generator.n//batch_size, verbose=1)


<ipython-input-41-207e94278253>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  preds = model.predict_generator(test_generator, steps=test_generator.n//batch_size, verbose=1)


2/2 [==============================] - 230s 113s/step


In [44]:
threshold = 0.5
for i in range(len(preds)):
    if preds[i] >= threshold:
        print(test_generator.filenames[i], "dysfunctional")
    else:
        print(test_generator.filenames[i], "functional")

1-112-PR-PID-005_1_3_resized.jpg functional
1-112-PR-PID-014_1_3_resized.jpg functional
9049679-120-13-PID Rev-01_resized.jpg dysfunctional
1-110-PR-PID-011_2_3_resized.jpg functional
9049679-120-14-PID Rev-01_resized.jpg functional
1-112-PR-PID-015_1_3_resized.jpg functional
9049679-120-15-PID Rev-01_resized.jpg dysfunctional
9049679-120-12-PID Rev-01_resized.jpg functional
1-110-PR-PID-001_2_3_resized.jpg functional
1-112-PR-PID-004_1_3_resized.jpg functional
9049679-120-09-PID Rev-01_resized.jpg dysfunctional
1-112-PR-PID-006_1_3_resized.jpg functional
1-110-PR-PID-003_2_3_resized.jpg functional
9049679-120-07-PID Rev-01_resized.jpg functional
9049679-120-16-PID-Rev-02_resized.jpg dysfunctional
1-110-PR-PID-002_2_3_resized.jpg functional
